In [ ]:
!pip install torch -Uq
!pip install transformers -Uq
!pip install tiktoken -Uq
!pip install datasets -Uq
!pip install matplotlib -Uq

In [ ]:
import json
import torch
import torch.nn as nn
import plotly.graph_objects as go
import plotly.offline

In [ ]:
class Tokenizer:
  def __init__(self, vocab_path):
    with open(vocab_path, "r", encoding="utf-8") as f:
      self.vocab = json.load(f)
      self.reverse_vocab = {v: k for k, v in self.vocab.items()}

  def encode(self, text):
    tokens = []

    for word in text.split():
      i = 0
      while i < len(word):
        found_match = False
        for j in range(len(word), i, -1):
          subword = word[i:j]
          if subword in self.vocab:
            tokens.append(self.vocab[subword])
            i = j
            found_match = True
            break
        if not found_match:
          tokens.append(self.vocab.get("<unk>", 0))
          i += 1
      tokens.append(self.vocab.get(" ", 1))
    if tokens:
      tokens.pop()
    return tokens

  def tokenize(self, text):
    token_ids = self.encode(text)
    return [self.reverse_vocab[id] for id in token_ids]

  def decode(self, ids):
    text = ""
    for id in ids:
      text += self.reverse_vocab.get(id, "<unk>")
    return text

In [ ]:
def get_rotary_position_encoding(input: torch.Tensor, base=10000, device="cpu"):
  context_length, dimension = input.shape

  assert dimension % 2 == 0, "boyutlar eşit olmalıdır"

  half_dimension = dimension // 2

  freqs_indices = torch.arange(0, half_dimension, device=device, dtype=torch.float32)
  freqs = 1.0 / (base ** (freqs_indices / dimension))

  positions = torch.arange(0, context_length, device=device, dtype=torch.float32).unsqueeze(1)

  angles = positions * freqs

  sin_angles = torch.sin(angles)
  cos_angles = torch.cos(angles)

  input_even = input[:, :dimension // 2]
  input_odd = input[:, dimension // 2:]

  input_even_rotated = input_even * cos_angles - input_odd * sin_angles
  input_odd_rotated = input_even * sin_angles + input_odd * cos_angles

  input_rotated = torch.empty_like(input)

  input_rotated[:, :dimension // 2] = input_even_rotated
  input_rotated[:, dimension // 2:] = input_odd_rotated

  return input_rotated

In [ ]:
class UstaModel(nn.Module):
  def __init__(self, vocab_size, embedding_dim, context_length):
    super().__init__()
    self.embedding = nn.Embedding(vocab_size, embedding_dim)

    self.get_pos = get_rotary_position_encoding

  def forward(self, x):
    x = self.embedding(x)
    x = self.get_pos(x)
    return x

In [ ]:
def plot_dots(sentences_data, title, dims=[0, 1, 2]):
  data = [
    go.Scatter3d(
      x=sentence_data["words"][:, dims[0]],
      y=sentence_data["words"][:, dims[1]],
      z=sentence_data["words"][:, dims[2]],
      mode="markers+text",
      marker=dict(
        size=6,
        color=sentence_data["color"],
      ),
      text=sentence_data["labels"],
      hoverinfo="text",
    ) for sentence_data in sentences_data
  ]

  layout = go.Layout(
    scene=dict(
      xaxis_title="Hardness",
      yaxis_title="Brightness",
      zaxis_title="Redness",
    ),
    title=title,
    width=1000,
    height=1000,
  )

  fig = go.Figure(data=data, layout=layout)
  plotly.offline.iplot(fig)

In [ ]:
tokenizer = Tokenizer("tokenizer_1.json")

In [ ]:
prompt = "the capital of united states and the capital of france"

In [ ]:
tokens = tokenizer.encode(prompt)

In [ ]:
x = torch.tensor(tokens, dtype=torch.long)

In [ ]:
torch.manual_seed(42)

In [ ]:
model = UstaModel(vocab_size=len(tokenizer.vocab), embedding_dim=4, context_length=32)

In [ ]:
model(x)

In [ ]:
sentence_meanings = model(x)

In [ ]:
sentence_meanings.shape

torch.Size([20, 4])

In [ ]:
sentences = [
  {
    "words": sentence_meanings.detach().numpy(),
    "labels": tokenizer.tokenize(prompt),
    "color": "red",
  },
]

plot_dots(sentences, "Models Context Space")

In [ ]:
sentence_meanings

In [ ]:
all_similarities = torch.zeros(sentence_meanings.shape[0], sentence_meanings.shape[0])
for j in range(sentence_meanings.shape[0]):
  j_similarities = torch.zeros(sentence_meanings.shape[0])

  for i in range(sentence_meanings.shape[0]):
    for k in range(sentence_meanings.shape[1]):
      cs_j_i = sentence_meanings[j][k] * sentence_meanings[i][k]
      j_similarities[i] += cs_j_i

  all_similarities[j] = j_similarities

In [ ]:
attention_weights = torch.softmax(all_similarities, dim=1)

In [ ]:
all_similarities.detach().numpy()

In [ ]:
'''class UstaSelfAttention(nn.Module):
  def __init__(self, embedding_dim, output_dim):
    super().__init__()
    self.embedding_dim = embedding_dim

    self.q_weights = nn.Linear(embedding_dim, output_dim, bias=False)
    self.k_weights = nn.Linear(embedding_dim, output_dim, bias=False)
    self.v_weights = nn.Linear(embedding_dim, output_dim, bias=False)

  def forward(self, x):
    q = self.q_weights(x)
    k = self.k_weights(x)
    v = self.v_weights(x)

    attention_scores = q @ k.T
    attention_weights = torch.softmax(attention_scores / k.shape[-1] ** 0.5, dim=1)
    return attention_weights @ v'''

In [ ]:
'''class UstaModel(nn.Module):
  def __init__(self, vocab_size, embedding_dim, context_length):
    super().__init__()

    self.embedding = nn.Embedding(vocab_size, embedding_dim)
    self.pos_embedding = nn.Embedding(context_length, embedding_dim)
    self.get_pos = get_rotary_position_encoding
    self.self_attention = UstaSelfAttention(embedding_dim, embedding_dim)

  def forward(self, x):
    x = self.embedding(x)
    x = self.get_pos(x)
    x = self.self_attention(x)
    return x'''